# Chapter 2. Poisson Processes

In [1]:
# Import packages.
import numpy as np

## Example 2.1 (Exponential Races) 

Anne and Betty enter a beauty parlor simultaneously, Anne to get a manicure and Betty to get a haircut. Suppose the time for a manicure (haircut) is exponentially distributed with mean 20 (30) minutes.

__(a)__ What is the probability Anne gets done first?

__(b)__ What is the expected amount of time until Anne and Betty are both done?

In [2]:
# Define the competing rates for Anne and Betty.
lambda_A = 1 / 20  # Anne's rate in minutes^{-1}.
lambda_B = 1 / 30  # Betty's rate in minutes^{-1}.


### Part (a): Probability Anne finishes first. ###

# Equation (2.8).
prob_A_first = lambda_A / (lambda_A + lambda_B)
# Print the result.
print(f"Part (a): Probability Anne finishes first = {prob_A_first}")


### Part (b): Expected total time until both are done. ###

# Mean time until first service completion.
mean_first_completion = 1 / (lambda_A + lambda_B)  # In minutes.
# Expected remaining time given Anne or Betty finishes first.
# First term: probability Anne finishes first times expected waiting time for Betty to finish her haircut.
# Second term: probability Betty finishes first times expected waiting time for Anne to finish her manicure.
expected_remaining_time = (prob_A_first * (1 / lambda_B)) + (
    (1 - prob_A_first) * (1 / lambda_A)
)
# Total expected time for Anne and Betty to finish their services.
total_expected_time = mean_first_completion + expected_remaining_time
# Print the result.
print(
    f"Part (b): Expected total time until both are done = {total_expected_time} minutes"
)

Part (a): Probability Anne finishes first = 0.6
Part (b): Expected total time until both are done = 38.0 minutes


## Example 2.2 (Exponential Races)

Ron, Sue, and Ted arrive at the beginning of a professor's office hours. The amount of time they will stay is exponentially distributed with means of $1, \frac{1}{2}$, and $\frac{1}{3}$ hour.

__(a)__ For each student find the probability they are the last student left. 

__(b)__ What is the expected time until all three students are gone?

In [18]:
# Define rates
lambda_R = 1  # Ron's rate (1/hr).
lambda_S = 2  # Sue's rate (2/hr).
lambda_T = 3  # Ted's rate (3/hr).


### Part (a): Probability that each student is last. ###

# Total rate when all three are present.
total_rate = lambda_R + lambda_S + lambda_T
# Probabilities of each student leaving first.
prob_R_first = lambda_R / total_rate
prob_S_first = lambda_S / total_rate
prob_T_first = lambda_T / total_rate
# Conditional probabilities of each student being the last to leave from each pair of students.
prob_R_last_from_RS = lambda_S / (lambda_R + lambda_S)
prob_R_last_from_RT = lambda_T / (lambda_R + lambda_T)
prob_S_last_from_RS = lambda_R / (lambda_R + lambda_S)
prob_S_last_from_ST = lambda_T / (lambda_S + lambda_T)
prob_T_last_from_RT = lambda_R / (lambda_R + lambda_T)
prob_T_last_from_ST = lambda_S / (lambda_S + lambda_T)
# Probability of each student being the last to leave.
prob_R_last = prob_S_first * prob_R_last_from_RT + prob_T_first * prob_R_last_from_RS
prob_S_last = prob_R_first * prob_S_last_from_ST + prob_T_first * prob_S_last_from_RS
prob_T_last = prob_R_first * prob_T_last_from_ST + prob_S_first * prob_T_last_from_RT
# Print the results.
print(f"Part (a): Probability Ron (R) is last = {prob_R_last:.3f}")
print(f"Part (a): Probability Sue (S) is last = {prob_S_last:.3f}")
print(f"Part (a): Probability Ted (T) is last = {prob_T_last:.3f}")


### Part (b): Expected time until all three students are gone. ###

# Expected waiting times until a student leaves.
mean_time_first_leave = 1 / total_rate
mean_time_RS_leave = 1 / (lambda_R + lambda_S)
mean_time_RT_leave = 1 / (lambda_R + lambda_T)
mean_time_ST_leave = 1 / (lambda_S + lambda_T)
# Expected time until all three students are gone.
expected_total_time = (
    mean_time_first_leave
    + prob_T_first * mean_time_RS_leave
    + prob_S_first * mean_time_RT_leave
    + prob_R_first * mean_time_ST_leave
    + prob_R_last * (1 / lambda_R)
    + prob_S_last * (1 / lambda_S)
    + prob_T_last * (1 / lambda_T)
)
# Print the result.
print(
    f"Part (b): Expected total time until all students are gone: {expected_total_time:.4f} hours"
)

Part (a): Probability Ron (R) is last = 0.583
Part (a): Probability Sue (S) is last = 0.267
Part (a): Probability Ted (T) is last = 0.150
Part (b): Expected total time until all students are gone: 1.2167 hours


## Example 2.3 (Compound Poisson Process)

Consider the McDonald's restaurant on Route 13 in the southern part of Ithaca. By arguments in the last section, it is not unreasonable to assume that between 12:00 and 1:00 cars arrive according to a Poisson process with rate $\lambda$. Let $Y_{i}$ be the number of people in the $i^{th}$ vehicle. There might be some correlation between the number of people in the car and the arrival time, but for a first approximation it seems reasonable to assume that the $Y_{i}$ are i.i.d. and independent of the Poisson process of arrival time. __Note:__ this is a motivating example in the book, but here is a solution to help motivate compound Poisson processes.

### Solution

The arrival of cars is modeled as a Poisson process with rate $\lambda$. If $\lambda$ is given in units of cars per hour, then the number of cars arriving between 12:00 and 1:00 (denoted by $N$) follows a Poisson distribution:
$$
\mathbb{P}\left( N = n \right) = e^{- \lambda} \frac{\lambda^{n}}{n!} \; .
$$ 
It is assumed that $Y_{i}$ are identically distributed, independent of each other, and independent of the Poisson process governing the car arrivals. This means $Y_{i}$ are i.i.d. with some distribution. For simplicity, let's assume $Y_{i}$ also follows a Poisson distribution with rate $\mu$. The total number of people arriving at the restaurant between 12:00 and 1:00 can be expressed as:
$$
S = \sum_{i = 1}^{N} Y_{i} \; ,
$$
where $N$ is Poisson($\lambda$) and each $Y_{i}$ is i.i.d. Poisson($\mu$). Given $N$ is Poisson and $Y_{i}$ are i.i.d. Poisson and independent of $N$, the sum $S$ also follows a Poisson distribution. The expected value of $S$ is the product of the expected values of $N$ and $Y_{i}$:
$$
\mathbb{E}\left[ S \right] = \mathbb{E}\left[ N \right] \cdot \mathbb{E}\left[ Y_{i} \right] = \lambda \mu \; .
$$
Similarly, since $N$ and $Y_{i}$ are independent and Poisson distributed the variance of $S$ is the product of the variance of $N$ and the second moment of $Y_{i}$:
$$
\mathbb{V}\left[ S \right] = \mathbb{E}\left[ N \right] \cdot \mathbb{V}\left[ Y_{i} \right] + \mathbb{V}\left[ N \right] \left( \mathbb{E}\left[ Y_{i} \right] \right)^{2} = \lambda \left(\mathbb{V}\left[ Y_{i} \right] +  \left( \mathbb{E}\left[ Y_{i} \right] \right)^{2} \right) = \lambda \left(\mu + \mu^{2} \right) \; ,
$$
reflecting the scaling of individual term effects by the rate of the Poisson process. Thus, $S$ is Poisson distributed with rate $\lambda \mu$.

In [29]:
# Initialize random seed.
np.random.seed(37)  # https://grossack.site/2023/11/08/37-median.html
# Initialize rates.
lambda_ = 20  # Average number of cars per hour.
mu = 1.5  # Average number of people per car.
# Initialize lists to store the simulated replicates.
Ns = []
Y_is = []
Ss = []
# Run the simulation one million times (this took me ~15 seconds).
for _ in range(1_000_000):
    # Simulate the number of cars arriving in one hour.
    N = np.random.poisson(lambda_)
    # Simulate the number of people in each car.
    Y_i = np.random.poisson(mu, N)
    # Calculate the total number of people arriving in one hour.
    S = np.sum(Y_i)
    # Update the simulated replicate lists.
    Ns.append(N)
    Y_is.extend(Y_i)
    Ss.append(S)
# Compare the analytical solutions to the simulation results.
print(
    f"Number of cars: E_{{N}} = {lambda_} vs mu_{{N}} = {np.mean(Ns):.4f} & V_{{N}} = {lambda_} vs sigma_squared_{{N}} = {np.var(Ns):.4f}"
)
print(
    f"People in each car: E_{{Y_i}} = {mu} vs mu_{{Y_i}} = {np.mean(Y_is):.4f} & V_{{Y_i}} = {mu} vs sigma_squared_{{Y_i}} = {np.var(Y_is):.4f}"
)
print(
    f"Total number of people: E_{{S}} = {lambda_ * mu} vs mu_{{S}} = {np.mean(Ss):.4f} & V_{{S}} = {lambda_ * (mu + mu**2)} vs sigma_squared_{{S}} = {np.var(Ss):.4f}"
)

Number of cars: E_{N} = 20 vs mu_{N} = 19.9951 & V_{N} = 20 vs sigma_squared_{N} = 19.9991
People in each car: E_{Y_i} = 1.5 vs mu_{Y_i} = 1.5000 & V_{Y_i} = 1.5 vs sigma_squared_{Y_i} = 1.5003
Total number of people: E_{S} = 30.0 vs mu_{S} = 29.9918 & V_{S} = 75.0 vs sigma_squared_{S} = 75.0999


## Example 2.4 (Compound Poisson Process)

Messages arrive at a computer to be transmitted across the Internet. If we imagine a large number of users writing emails on their laptops (or tablets or smart phones) then the arrival times of messages can be modeled by a Poisson process. If we let $Y_{i}$ be the size of the $i^{th}$ message, then again it is reasonable to assume $Y_{1}, Y_{2}, \ldots$ are i.i.d. and independent of the Poisson process of arrival times. __Note:__ this is a motivating example in the book, but here is a solution to help motivate compound Poisson processes.

### Solution

The arrival of messages is modeled as a Poisson process with rate $\lambda$, which could be given in units like messages per hour. If we consider a specific time window, such as one hour, the number of messages $N$ that arrive during this period is Poisson distributed:
$$
\mathbb{P}\left( N = n \right) = e^{- \lambda} \frac{\lambda^{n}}{n!} \; .
$$ 
It is assumed that $Y_{i}$ are identically distributed, independent of each other, and independent of the Poisson process governing the car arrivals. This means $Y_{i}$ are i.i.d. with some distribution. For simplicity, let's assume $Y_{i}$ follows a Log-normal distribution with parameters $\mu$ and $\sigma$ (the mean and standard deviation of the underlying normal distribution) to better reflect the variability and skewness typically observed in message sizes. The total size of the messages for an hour time interval can be expressed as:
$$
S = \sum_{i = 1}^{N} Y_{i} \; ,
$$
where $N$ is Poisson($\lambda$) and each $Y_{i}$ is i.i.d. Similar to the previous example the expected value and variance of $S$ are:
$$
\mathbb{E}\left[ S \right] = \mathbb{E}\left[ N \right] \cdot \mathbb{E}\left[ Y_{i} \right] = \lambda e^{\mu + \frac{\sigma^{2}}{2}} \; ,
$$
$$
\mathbb{V}\left[ S \right] = \mathbb{E}\left[ N \right] \cdot \mathbb{V}\left[ Y_{i} \right] + \mathbb{V}\left[ N \right] \left( \mathbb{E}\left[ Y_{i} \right] \right)^{2} = \lambda \left( \left( e^{\sigma^{2}} - 1 \right) e^{2 \mu + \sigma^{2}} \right) + \lambda \left( e^{\mu + \frac{\sigma^{2}}{2}} \right)^{2} = \lambda e^{2 \mu +  2\sigma^{2}} \; .
$$

In [31]:
# Initialize random seed.
np.random.seed(37)  # https://grossack.site/2023/11/08/37-median.html
# Initialize rates.
lambda_ = 10  # Average number of messages per hour
mu = 1  # Mean of the log for the log-normal distribution (log scale).
sigma = (
    0.5  # Standard deviation of the log for the log-normal distribution (log scale).
)
# Initialize lists to store the simulated replicates.
Ns = []
Y_is = []
Ss = []
# Run the simulation one million times (this took me ~12 seconds).
for _ in range(1_000_000):
    # Simulate the number of messages arriving in one hour.
    N = np.random.poisson(lambda_)
    # # Simulate the sizes of each message.
    Y_i = np.random.lognormal(mu, sigma, N)
    # Calculate the total size of messages in one hour.
    S = np.sum(Y_i)
    # Update the simulated replicate lists.
    Ns.append(N)
    Y_is.extend(Y_i)
    Ss.append(S)
# Compare the analytical solutions to the simulation results.
print(
    f"Number of messages: E_{{N}} = {lambda_} vs mu_{{N}} = {np.mean(Ns):.4f} & V_{{N}} = {lambda_} vs sigma_squared_{{N}} = {np.var(Ns):.4f}"
)
print(
    f"Sizes of messages: E_{{Y_i}} = {np.exp(mu + ((sigma**2) / 2)):.4f} vs mu_{{Y_i}} = {np.mean(Y_is):.4f} & V_{{Y_i}} = {((np.exp(sigma**2) - 1) * np.exp((2 * mu) + (sigma**2))):.4f} vs sigma_squared_{{Y_i}} = {np.var(Y_is):.4f}"
)
print(
    f"Total size of messages: E_{{S}} = {(lambda_ * np.exp(mu + ((sigma**2) / 2))):.4f} vs mu_{{S}} = {np.mean(Ss):.4f} & V_{{S}} = {(lambda_ * np.exp((2 * mu) + (2 * (sigma**2)))):.4f} vs sigma_squared_{{S}} = {np.var(Ss):.4f}"
)

Number of messages: E_{N} = 10 vs mu_{N} = 10.0019 & V_{N} = 10 vs sigma_squared_{N} = 9.9921
Sizes of messages: E_{Y_i} = 3.0802 vs mu_{Y_i} = 3.0804 & V_{Y_i} = 2.6948 vs sigma_squared_{Y_i} = 2.6968
Total size of messages: E_{S} = 30.8022 vs mu_{S} = 30.8100 & V_{S} = 121.8249 vs sigma_squared_{S} = 121.7755


## Example 2.5 (Compound Poisson Process)

Suppose that the number of customers at a liquor store in a day has a Poisson distribution with mean 81 and that each customer spends an average of $\$ 8$ with a standard deviation of $\$ 6$.

__(a)__ Find the liquor store's expected daily revenue.

__(b)__ Find the variance of liquor store's daily revenue.

__(c)__ Find the standard deviation of liquor store's daily revenue.

In [33]:
# Initialize the liquor store's parameters.
mean_daily_customers = 81
mean_expenditure = 8
std_expenditure = 6


### Part (a): The liquor store's expected daily revenue. ###

# Theorem (2.10).
exp_daily_revenue = mean_daily_customers * mean_expenditure
# Print the result.
print(f"Part (a):  The liquor store's expected daily revenue = ${exp_daily_revenue}")


### Part (b): The variance of liquor store's daily revenue. ###

# Theorem (2.10).
var_daily_revenue = mean_daily_customers * (
    (std_expenditure**2) + (mean_expenditure**2)
)
# Print the result.
print(f"Part (b): The variance of liquor store's daily revenue = ${var_daily_revenue}")


### Part (c): The standard deviation of liquor store's daily revenue. ###
std_daily_revenue = np.sqrt(var_daily_revenue)
# Print the result.
print(
    f"Part (c): The standard deviation of liquor store's daily revenue = ${std_daily_revenue}"
)

Part (a):  The liquor store's expected daily revenue = $648
Part (b): The variance of liquor store's daily revenue = $8100
Part (c): The standard deviation of liquor store's daily revenue = $90.0
